# CSC321 Tutorial 11: Text Generation

In this tutorial, we will use recurrent neural networks to **generate** 
sequences. 
Generating sequences is more involved comparing to making predictions about
sequences. However, it is a very interesting task. This tutorial will help
prepare you for project 4.

Much of today's content is an adaptation of the "Practical PyTorch" github 
repository [1], and Lisa's notes on Generating Trump Tweets [2].

In [1]:
# [1] https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb
# [2] https://www.cs.toronto.edu/~lczhang/360/lec/w08/gen.html

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Reccurent Neural Networks Revisited

The recurrent neural network architecture from last time 
looked something like this:

<img src="https://www.cs.toronto.edu/~lczhang/321/tut/imgs/rnn.png" width="500px">

The input sequence is broken down into tokens. We could choose whether to
tokenize based on words, or based on characters. The representation
of each token (GloVe or one-hot) is processed by the RNN one step at a time
to update the hidden (or context) state.

In a predictive RNN, the value of the
hidden states  is a representation of **all the text that was processed thus far**.
Similarly, in a generative RNN, The value of the hidden
state will be a representation of **all the text that still needs to be generated**.
We will use this hidden state to produce the sequence, one token at a time.

Similar to last class we will break up the problem of generating text
to generating one token at a time.

We will do so with the help of two functions:

1. We need to be able to generate the *next* token, given the current 
   hidden state. In practice, we get a probability distribution over 
   the next token, and sample from that probability distribution.
2. We need to be able to update the hidden state somehow. To do so,
   we need two piece of information: the old hidden state, and the actual
   token that was generated in the previous step. The actual token generated
   will inform the subsequent tokens.

We will repeat both functions until a special "END OF SEQUENCE" token is
generated. Here is a pictorial representation of what we will do:

<img src="https://www.cs.toronto.edu/~lczhang/321/tut/imgs/rnn_gen_figure.png" width="500px">

Note that there are several tricky things that we will have to figure out.
For example, how do we actually sample the actual token from the probability
distribution over tokens? What would we do during training, and how might 
that be different from during testing/evaluation? We will answer those
questions as we implement the RNN.

For now, let's start with our training data.

## Data: Happy Tweets from 2018

The training set we use is a collection of "happy" tweets from the Sentiment140
data set.
We will only use tweets that are 140 characters or shorter, and tweets
that contains more than just a URL.
Since tweets often contain creative spelling and numbers, and upper vs lower
case characters are read very differently, we will use a character-level RNN.

In [3]:
import csv
tweets = list(line[0] for line in csv.reader(open('happy.csv', encoding="utf8")))
len(tweets)

20000

Let's look at a few of these tweets, just to get a sense of the kind of text
we're dealing with:

In [4]:
print(tweets[100])
print(tweets[1000])
print(tweets[10000])

@_hayles yeh!!!nowt wrong with my wee fingers  type type type.. lol oohh spray tan!never had 1 b4
@graemerawson Thanks. Probably looking for someone local
@michaelahills YEP 


## Generating One Tweet

Our generative RNN is a **generative model**. At a high level, generative
models are trained to maximize the probability of generating our training data.
We'll need to figure out what that means, and to manipulate our data 
Our RNN model generates text one character at a time. In each iteration,
we'll train our data to generate a different tweet.

Normally, when we build a new machine learn model, we want to make sure
that our model can overfit. To that end, we will first build a neural network
that can generate _one_ tweet really well. We can choose any tweet (or any other text)
we want.  Let's choose to build an RNN that generates `tweet[200]`.

In [5]:
tweet = tweets[200]
tweet

'@whin5 nice one '

Since tweets are often poorly spelled, we will treat a tweet as a **sequence of characters**.
Our RNN will generate one **character** as a time.

Since PyTorch works with numbers instead of strings, we will need to convert
characters into integers. This of this integer as a sparse representation of a
one-hot encoding.  We'll build dictionary mappings
from the character to the index of that character (a unique integer identifier),
and from the index to the character. We'll use the same naming scheme that `torchtext`
uses (`stoi` and `itos`, which means "string to index" and "index to string").

For simplicity, we'll work with a limited vocabulary containing
just the characters in `tweet[200]`, plus two special tokens:

- `<EOS>` represents "End of String", which we'll append to the end of our tweet.
  Since tweets are variable-length, this is a way for the RNN to signal
  that the entire sequence has been generated.
- `<BOS>` represents "Beginning of String", which we'll prepend to the beginning of 
  our tweet. This is the first token that we will feed into the RNN.

The way we use these special tokens will become more clear as we build the model.

### Part (a)

Build two Python dictionary mappings
from the character to the index of that character (a unique integer identifier),
and from the index to the character. We'll use the same naming scheme that `torchtext`
uses (`stoi` and `itos`).

In [6]:
vocab = list(set(tweet)) + ["<BOS>", "<EOS>"]
vocab_stoi = {} # TODO: build a dictionary mapping of word to unique index
vocab_stoi= {s: i for i, s in enumerate(vocab)}

vocab_itos = {} # TODO: build a dictionary mapping of a unique index to a word (string)
vocab_itos = {i: s for i, s in enumerate(vocab)}
vocab_size = len(vocab)

# DONE

In [7]:
vocab_stoi['<BOS>']

10

### Part (b)

Our model have three parts:

1. An **embedding** layer that takes a character index (sparse representation of a one-hot vector
   representing the character), and returns an **embedding** ${\bf x}^{(t)}$ for the character.
   We could use a one-hot embedding here, but using a low-dimensional but dense embedding
   is more powerful, and is more in line with what we will do in Project 4.
2. A **recurrent neural network** layer. We will use a `nn.GRU` unit. At each time step,
   this layer will take the previous hidden state ${\bf h}^{(t-1)}$ and the embedding of
   a new generated token ${\bf x}^{(t)}$ and compute the new hidden state ${\bf h}^{(t)}$ representing
   *tokens not yet generated*.
3. The **projection MLP** (a fully-connected layer) that takes the hidden state ${\bf h}^{(t)}$
   and computes teh distribution of the next character to generate.

Filling in the missing numbers in the `__init__` method using
the parameters `vocab_size`, `emb_size`, and `hidden_size`.

In [8]:
class TextGenerator(nn.Module):
    def __init__(self,
                 vocab_size,     # number of unique characters in our vocabulary
                 embedding_size, # size of the word embeddings ${\bf x}^{(t)}$
                 hidden_size):   # size of the hidden state in the RNN
        super(TextGenerator, self).__init__()

        # Embedding 
        self.embed = nn.Embedding(num_embeddings=vocab_size, # TODO
                                  embedding_dim=embedding_size)  # TODO

        # recurrent neural network
        self.rnn = nn.GRU(input_size=embedding_size, #TODO
                          hidden_size=hidden_size, #TODO
                          batch_first=True)

        # a fully-connect layer that outputs a distribution over
        # the next token, given the RNN output
        self.proj = nn.Linear(in_features=hidden_size, # TODO
                              out_features=vocab_size) # TODO

    def forward(self, inp, hidden=None):
        emb = self.embed(inp)                  # generate one-hot vectors of input
        output, hidden = self.rnn(emb, hidden) # get the next output and hidden state
        output = self.proj(output)             # predict distribution over next tokens
        return output, hidden

model = TextGenerator(vocab_size, 128, 128)

## Training with Teacher Forcing

At a very high level, we want our RNN model to have a high probability
of generating the tweet. An RNN model generates text
one character at a time based on the hidden state value.
At each time step, we will check whether the mdoel generated the
correct character. That is, at each time step,
we are trying to select the correct next character out of all the 
characters in our vocabulary. Recall that this problem is a multi-class
classification problem, and we can use Cross-Entropy loss to train our
network to become better at this type of problem.

In [9]:
criterion = nn.CrossEntropyLoss()

However, we don't just have a single multi-class classification problem.
Instead, we have **one classification problem per time-step** (per token)!
So, how do we predict the first token in the sequence? 
How do we predict the second token in the sequence? 

To help you understand what happens durign RNN training, we'll start with a
inefficient training code that shows you what happens step-by-step. We'll
start with computing the loss for the first token generated, then the second token,
and so on.
Later on, we'll switch to a simpler and more performant version of the code.

So, let's start with the first classification problem: the problem of generating
the **first** token (`tweet[0]`).

<img src="https://www.cs.toronto.edu/~lczhang/321/tut/imgs/rnn_gen_figure.png" width="500px">

To generate the first token, we'll feed the RNN network (with an initial, empty
hidden state) the "<BOS>" token. Then, the output

In [10]:
bos_input = torch.Tensor([vocab_stoi["<BOS>"]]).long().unsqueeze(0)
output, hidden = model(bos_input, hidden=None)
output # distribution over the first token

tensor([[[-0.0022,  0.4379,  0.0921,  0.0174,  0.1430, -0.0266, -0.0144,
          -0.0998, -0.1854,  0.1171,  0.0660, -0.2929]]],
       grad_fn=<AddBackward0>)

We can compute the loss using `criterion`. Since the model is untrained,
the loss is expected to be high. (For now, we won't do anything
with this loss, and omit the backward pass.)

In [11]:
target = torch.Tensor([vocab_stoi[tweet[0]]]).long().unsqueeze(0)
criterion(output.reshape(-1, vocab_size), # reshape to 2D tensor
          target.reshape(-1))             # reshape to 1D tensor

tensor(2.5042, grad_fn=<NllLossBackward>)

Now, we need to update the hidden state and generate a prediction
for the next token. To do so, we need to provide the current token to
the RNN. We already said that during test time, we'll need to sample
from the predicted probabilty over tokens that the neural network
just generated. 

Right now, we can do something better: we can **use the ground-truth,
actual target token**. This technique is called **teacher-forcing**, 
and generally speeds up training. The reason is that right now, 
since our model does not perform well, the predicted probability
distribution is pretty far from the ground truth. So, it is very,
very difficult for the neural network to get back on track given bad
input data.

In [12]:
# Use teacher-forcing: we pass in the ground truth `target`,
# rather than using the NN predicted distribution
output, hidden = model(target, hidden)
output # distribution over the second token

tensor([[[ 0.1702,  0.1959, -0.0536, -0.2001,  0.0015, -0.0085, -0.0152,
          -0.1355, -0.1310,  0.1309, -0.1383, -0.1810]]],
       grad_fn=<AddBackward0>)

Similar to the first step, we can compute the loss, quantifying the
difference between the predicted distribution and the actual next
token. This loss can be used to adjust the weights of the neural
network (which we are not doing yet).

In [13]:
target = torch.Tensor([vocab_stoi[tweet[1]]]).long().unsqueeze(0)
criterion(output.reshape(-1, vocab_size), # reshape to 2D tensor
          target.reshape(-1))             # reshape to 1D tensor

tensor(2.4716, grad_fn=<NllLossBackward>)

We can continue this process of:

- feeding the previous ground-truth token to the RNN,
- obtaining the prediction distribution over the next token, and
- computing the loss,

for as many steps as there are tokens in the ground-truth tweet.

In [14]:
for i in range(2, len(tweet)):
    output, hidden = model(target, hidden)
    target = torch.Tensor([vocab_stoi[tweet[1]]]).long().unsqueeze(0)
    loss = criterion(output.reshape(-1, vocab_size), # reshape to 2D tensor
                     target.reshape(-1))             # reshape to 1D tensor
    print(i, output, loss)

2 tensor([[[ 0.2488,  0.2693,  0.0383,  0.1714,  0.0561, -0.0518, -0.0662,
          -0.0547, -0.2378, -0.1083, -0.0794, -0.1511]]],
       grad_fn=<AddBackward0>) tensor(2.5514, grad_fn=<NllLossBackward>)
3 tensor([[[ 0.2772,  0.2910,  0.1358,  0.3478,  0.0824, -0.0323, -0.1525,
          -0.0234, -0.2543, -0.2274, -0.0347, -0.1526]]],
       grad_fn=<AddBackward0>) tensor(2.5584, grad_fn=<NllLossBackward>)
4 tensor([[[ 0.2848,  0.2975,  0.2124,  0.4299,  0.0926, -0.0137, -0.2117,
          -0.0090, -0.2483, -0.2883, -0.0066, -0.1618]]],
       grad_fn=<AddBackward0>) tensor(2.5559, grad_fn=<NllLossBackward>)
5 tensor([[[ 0.2842,  0.2996,  0.2646,  0.4682,  0.0951,  0.0020, -0.2482,
          -0.0028, -0.2408, -0.3215,  0.0091, -0.1726]]],
       grad_fn=<AddBackward0>) tensor(2.5489, grad_fn=<NllLossBackward>)
6 tensor([[[ 0.2805,  0.3011,  0.2982,  0.4861,  0.0949,  0.0154, -0.2704,
          -0.0012, -0.2360, -0.3403,  0.0169, -0.1832]]],
       grad_fn=<AddBackward0>) tensor(2.540

Finally, with our final token, we should expect to output the "<EOS>"
token, so that our RNN learns when to stop generating characters.

In [15]:
output, hidden = model(target, hidden)
target = torch.Tensor([vocab_stoi["<EOS>"]]).long().unsqueeze(0)
loss = criterion(output.reshape(-1, vocab_size), # reshape to 2D tensor
                 target.reshape(-1))             # reshape to 1D tensor
print(i, output, loss)

15 tensor([[[ 0.2545,  0.3152,  0.3603,  0.4967,  0.0856,  0.0589, -0.3057,
          -0.0155, -0.2286, -0.3650,  0.0171, -0.2246]]],
       grad_fn=<AddBackward0>) tensor(2.7830, grad_fn=<NllLossBackward>)


In practice, we don't really need a loop. Recall that in a predictive RNN,
the `nn.RNN` module can take an entire sequence as input. We can do the
same thing here:

In [16]:
list(tweet)

['@',
 'w',
 'h',
 'i',
 'n',
 '5',
 ' ',
 'n',
 'i',
 'c',
 'e',
 ' ',
 'o',
 'n',
 'e',
 ' ']

In [17]:
tweet_ch = ["<BOS>"] + list(tweet) + ["<EOS>"]
tweet_indices = [vocab_stoi[ch] for ch in tweet_ch]
tweet_tensor = torch.Tensor(tweet_indices).long().unsqueeze(0)

print(tweet_tensor.shape)

output, hidden = model(tweet_tensor[:,:-1]) # <EOS> is never an input token
target = tweet_tensor[:,1:]                 # <BOS> is never a target token
loss = criterion(output.reshape(-1, vocab_size), # reshape to 2D tensor
                 target.reshape(-1))             # reshape to 1D tensor

torch.Size([1, 18])


In [121]:
tweet_tensor[:,:-1]
# target

tensor([[10,  3,  5,  2,  9,  6,  8,  0,  6,  9,  1,  4,  0,  7,  6,  4,  0]])

Here, the input to our neural network model is the *entire*
sequence of input tokens (everything from "<BOS>" to the
last character of the tweet). The neural network generates a prediction distribution
of the next token at each step. We can compare each of these  with the ground-truth
`target`.

### Part (c)

Complete the code for the training loop to generate a single tweet.

In [21]:
tweet

'@whin5 nice one '

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for it in range(1000): # You might need to increase this
    optimizer.zero_grad()
    output, hidden = model(tweet_tensor[:,:-1]) # TODO
    loss = criterion(output.reshape(-1, vocab_size),
                 target.reshape(-1))      # TODO
    loss.backward()
    optimizer.step()

    if (it+1) % 100 == 0:
        print("[Iter %d] Loss %f" % (it+1, float(loss)))

[Iter 100] Loss 0.012211
[Iter 200] Loss 0.004100
[Iter 300] Loss 0.002179
[Iter 400] Loss 0.001382
[Iter 500] Loss 0.000966
[Iter 600] Loss 0.000717
[Iter 700] Loss 0.000555
[Iter 800] Loss 0.000444
[Iter 900] Loss 0.000363
[Iter 1000] Loss 0.000302


Once this code runs, your training code should decrease with training,
which is what we expect.

## Generating a Token

At this point, we want to see whether our model is actually learning
something. So, we need to talk about how to
actually use the RNN model to generate text. If we can 
generate text, we can make a qualitative asssessment of how well
our RNN is performing.

The main difference between training and test-time (generation time)
is that we don't have the ground-truth tokens to feed as inputs
to the RNN. Instead, we need to actually **sample** a token based
on the neural network's prediction distribution.

But how can we sample a token from a distribution?

On one extreme, we can always take
the token with the largest probability (argmax). This has been our
go-to technique in other classification tasks. However, this idea
will fail here. The reason is that in practice, 
**we want to be able to generate a variety of different sequences from
the same model**. An RNN that can only generate a single new tweet
is fairly useless.

In short, we want some randomness. We can do so by using the logit
outputs from our model to construct a multinomial distribution over
the tokens, then and sample a random token from that multinomial distribution.

One natural multinomial distribution we can choose is the 
distribution we get after applying the softmax on the outputs.
However, we will do one more thing: we will add a **temperature**
parameter to manipulate the softmax outputs. We can set a
**higher temperature** to make the probability of each token
**more even** (more random), or a **lower temperature** to assighn
more probability to the tokens with a higher logit (output).
A **higher temperature** means that we will get a more diverse sample,
with potentially more mistakes. A **lower temperature** means that we
may see repetitions of the same high probability sequence.

In [94]:
def sample_sequence(model, max_len=100, temperature=0.8):
    generated_sequence = ""
   
    inp = torch.Tensor([vocab_stoi["<BOS>"]]).long()
    hidden = None
    for p in range(max_len):
        output, hidden = model(inp.unsqueeze(0), hidden)
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i]
        
        if predicted_char == "<EOS>":
            break
        generated_sequence += predicted_char       
        inp = torch.Tensor([top_i]).long()
    return generated_sequence

In [98]:
"input is n"
def sample_sequence2(model, max_len=100, temperature=0.8):
    generated_sequence = ""
   
    inp = torch.Tensor([vocab_stoi["n"]]).long()
    hidden = None
    for p in range(max_len):
        output, hidden = model(inp.unsqueeze(0), hidden)
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i]
        
        if predicted_char == "<EOS>":
            break
        generated_sequence += predicted_char       
        inp = torch.Tensor([top_i]).long()
    return generated_sequence

In [102]:
'''input to trained model is: n '''
print('1.0 :',sample_sequence2(model, temperature=1.0))
print('1.5 :',sample_sequence2(model, temperature=1.5))
print('0.5 :',sample_sequence2(model, temperature=0.5))
print('0.25 :',sample_sequence2(model, temperature=0.25))
print('2 :', sample_sequence2(model, temperature=2.0))
print('3 :',sample_sequence2(model, temperature=3.0))
print('4 :',sample_sequence2(model, temperature=4.0))
print('5 :',sample_sequence2(model, temperature=5.0))

1.0 : e 
1.5 : e 
0.5 : 5 nice one 
0.25 : 5 nice one 
2 : e 
3 : 5@whin5@
4 :  cne @
5 : @cw


## Part (d)

Try sampling from your model with different temperature settings
(e.g. from 0.8 to 5). Make sure to sample multiple times per setting.
Since we only trained the model on a single sequence, we won't see
the full effect of the temperature parameter yet.

In [103]:
"input is <BOS>"
print('1.0 :',sample_sequence(model, temperature=1.0))
print('1.5 :',sample_sequence(model, temperature=1.5))
print('0.5 :',sample_sequence(model, temperature=0.5))
print('0.25 :',sample_sequence(model, temperature=0.25))
print('2 :', sample_sequence(model, temperature=2.0))
print('3 :',sample_sequence(model, temperature=3.0))
print('4 :',sample_sequence(model, temperature=4.0))
print('5 :',sample_sequence(model, temperature=5.0))

1.0 : @whin5 nice one 
1.5 : @whin5 nice one 
0.5 : @whin5 nice one 
0.25 : @whin5 nice one 
2 : @whin5 noce no@whinwh
3 : @whiii whe nc<BOS>e 
4 : @cen o<BOS>@whwhin5ce<BOS> oehe 
5 : h<BOS>oineownn


For now, the output of the calls to the `sample_sequence` function
assures us that our training code looks reasonable, and we can
proceed to training on our full dataset!

## Training the Tweet Generator

For the actual training, let's use `torchtext` so that we can use
the `BucketIterator` to make batches. Like in Lab 5, we'll create a 
`torchtext.data.Field` to use `torchtext` to read the CSV file, and convert
characters into indices. The object has convient parameters to specify
the BOS and EOS tokens.

In [104]:
import torchtext

text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True,       # to turn each character into an integer index
                                  init_token="<BOS>",   # BOS token
                                  eos_token="<EOS>")    # EOS token

fields = [('text', text_field)]
tweets = torchtext.data.TabularDataset("happy.csv", "csv", fields)
len(tweets)

20000

We will use the version of `vocab_stoi` and `vocab_itos` that torchtext provides.

In [105]:
text_field.build_vocab(tweets)
vocab_stoi = text_field.vocab.stoi # so we don't have to rewrite sample_sequence
vocab_itos = text_field.vocab.itos # so we don't have to rewrite sample_sequence
vocab_size = len(text_field.vocab.itos)
vocab_size

180

In [111]:
for k, v in vocab_stoi.items():
    if v >= 10: break
    print(k,v)

<unk> 0
<pad> 1
<BOS> 2
<EOS> 3
  4
e 5
o 6
t 7
a 8
i 9


Let's just verify that the `BucketIterator` works as expected, but start with batch_size of 1.

In [112]:
data_iter = torchtext.data.BucketIterator(tweets, 
                                          batch_size=1,
                                          sort_key=lambda x: len(x.text),
                                          sort_within_batch=True)
for (tweet, lengths), label in data_iter:
    print(label)   # should be None
    print(lengths) # contains the length of the tweet(s) in batch
    print(tweet.shape) # should be [1, max(length)]
    break

None
tensor([52])
torch.Size([1, 52])


## Part (e)

Modify the training code to account for batching.

In [113]:
def train(model, data, batch_size=1, num_epochs=1, lr=0.001, print_every=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    it = 0
    
    data_iter = torchtext.data.BucketIterator(data,
                                              batch_size=batch_size,
                                              sort_key=lambda x: len(x.text),
                                              sort_within_batch=True)
    for e in range(num_epochs):
        # get training set
        avg_loss = 0
        for (tweet, lengths), label in data_iter:
            target = tweet[:, 1:] # TODO
            inp = tweet[:, :-1] # TODO
            # cleanup
            optimizer.zero_grad()
            # forward pass
            output, _ = model(inp) # TODO
            loss = criterion(output.reshape(-1, vocab_size), target.reshape(-1)) # TODO
            # backward pass
            loss.backward()
            optimizer.step()

            avg_loss += loss
            it += 1 # increment iteration count
            if it % print_every == 0:
                print("[Iter %d] Loss %f" % (it+1, float(avg_loss/print_every)))
                print("    " + sample_sequence(model, 140, 0.8))
                avg_loss = 0

model = TextGenerator(vocab_size, 128, 128)

## Part (f)

Train your model. This model might take a looong time too train. However, the model
learns very quickly that many tweets begin with `@`.

In [114]:
train(model, tweets, batch_size=1, num_epochs=1, lr=0.004, print_every=100)
#train(model, tweets, batch_size=32, num_epochs=1, lr=0.004, print_every=100)

[Iter 101] Loss 3.188071
    @ind Iot han.. Yat ntenil... win.aother thing Won anucoy wahiven  hating tusss atthaye  oun wat ar daa an Ther sra hing Ee pot wingil.. 
[Iter 201] Loss 2.907591
    @jbai bea
[Iter 301] Loss 2.754739
    @Rekiorninin ge anthe ons &qukinings ap stoplerulk
[Iter 401] Loss 2.723065
    mingerd
[Iter 501] Loss 2.642127
    @LOM Ohs 
[Iter 601] Loss 2.604728
    @lazby yout gooda
[Iter 701] Loss 2.594453
    @guata to the 
[Iter 801] Loss 2.536493
    Ho
[Iter 901] Loss 2.562485
    Hi
[Iter 1001] Loss 2.488791
    iiah bay and
[Iter 1101] Loss 2.469344
    @icarmorter bob ah er bood thelly 
[Iter 1201] Loss 2.444599
    ok st;y wooken the so to is hawing ther come ar tot chak... 
[Iter 1301] Loss 2.470910
    @makeardaytt1 be I coon't you selyoo
[Iter 1401] Loss 2.520337
    Ems
[Iter 1501] Loss 2.604313
    It's hat the  cot al diss arffr the rearse 
[Iter 1601] Loss 2.457621
    @sagaster (TH Hah 4' hudtywerse to Some co ceand 
[Iter 1701] Loss 2.536667
    

    man a abrest sice 
[Iter 10801] Loss 2.384819
    @mitt_hrirem soool
[Iter 10901] Loss 2.398401
    @gics wold is my the till a
[Iter 11001] Loss 2.342171
    not a fat
[Iter 11101] Loss 2.344911
    @sanheldissne sone 
[Iter 11201] Loss 2.392545
    @bedelR Thouling hisping tomore 
[Iter 11301] Loss 2.351994
    @AKandav Have Heyian lek havad! 
[Iter 11401] Loss 2.337790
    @ziceferryleeptersisca the kipes that funny
[Iter 11501] Loss 2.359103
    @Righirst 155555
[Iter 11601] Loss 2.363757
    @areed my laheshot hist than nevioh!   That's eahing 
[Iter 11701] Loss 2.342891
    @zisoi toooooo missy today  Thatiar Mocom! 
[Iter 11801] Loss 2.285158
    I heh as all si going
[Iter 11901] Loss 2.398196
    @pannnyane_chimi canhigre
[Iter 12001] Loss 2.333675
    Gothing. 
[Iter 12101] Loss 2.310496
    @JillMymienEan Gorning 
[Iter 12201] Loss 2.333087
    Off on
[Iter 12301] Loss 2.355722
    Yo i arite yourding to and cool kind night 
[Iter 12401] Loss 2.369042
    @vertlear
[Iter

## Part (g)

Try generating some sequences using your model. Vary your temperature settings.

In [ ]:
#print(sample_sequence(model, temperature=0.8))
#print(sample_sequence(model, temperature=0.8))
#print(sample_sequence(model, temperature=1.0))
#print(sample_sequence(model, temperature=1.0))
#print(sample_sequence(model, temperature=1.5))
#print(sample_sequence(model, temperature=1.5))
#print(sample_sequence(model, temperature=2.0))
#print(sample_sequence(model, temperature=2.0))
#print(sample_sequence(model, temperature=5.0))
#print(sample_sequence(model, temperature=5.0))